In [43]:
import socket
import threading
import os
import shutil

from tkinter import *
from tkinter import filedialog

In [44]:
IP_Servidor = '127.0.0.1'             
# Endereco IP do Servidor

PORTA_Servidor = 8000
# Porta em que o servidor estara ouvindo

tcp = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# socket.AF_INET = INET (exemplo IPv4)sockets, #socket.SOCK_STREAM=usaremos TCP

DESTINO = (IP_Servidor, PORTA_Servidor) 
#destino(IP + porta)

tcp.connect(DESTINO) 
# inicia a conexao TCP

testa_mensagem=''

user = {'apelido':'', 'server':'', 'cont':0} 

if os.path.isdir('./cliente') == False:
    path = os.path.join('./', 'cliente')
    os.mkdir(path)
else:
    shutil.rmtree('./cliente')
    path = os.path.join('./', 'cliente')
    os.mkdir(path)

In [45]:
def saveArquivo (conteudoComando,apelido,lbl,conversa):
  
  end = bytes("#enviado","utf8")

  if user['apelido'] != '':
    path = os.path.join('./'+apelido+'/'+conteudoComando[1])
  else:
    path = os.path.join('./cliente/'+conteudoComando[1])
  
  arq = open(path, 'wb')

  while 1:
      dados = tcp.recv(1024)
      if dados == end:
        print("\rrecebendo...")
        break
      arq.write(dados)
  arq.close()

  Mensagem = "O arquivo: "+conteudoComando[1]+"\nsalvo na pasta: "+path

  conversa.append(Mensagem)
  lbl.config(text = conversa)

  print("\rRecebido")

In [46]:
def recebendoMensagem(lbl,conversa):
  conteudoComando = []
  while 1:

    Mensagem = ''
    #Recebe mensagem
    Mensagem_Recebida = tcp.recv(1024)
    #Mensagem recebida do cliente 

    #aqui verifica se exite mensagem nova 
    try:
      Mensagem_Recebida = Mensagem_Recebida.decode("utf-8") 
      if testa_mensagem != Mensagem_Recebida:  
        conteudoComando = Mensagem_Recebida.split('|')
        
        match conteudoComando[0]:

          case '#arquivo':
            print('arquivo')
            saveArquivo(conteudoComando,user['apelido'],lbl,conversa)


          case '#apelido':
            user['apelido'] = conteudoComando[1]
            user['ip'] = DESTINO

            path = os.path.join('./', 'cliente')
            newPath = os.path.join('./', user['apelido'])
            os.rename(path,newPath)

            Mensagem = "Apelido recebido: "+ conteudoComando[1] +" \n"
            conversa.append(Mensagem)
            lbl.config(text = conversa)

          case _:
            if user['apelido'] != '':
              Mensagem = user['apelido'] +": "+ Mensagem_Recebida+"\n"
              
              conversa.append(Mensagem)
              lbl.config(text = conversa)
            else:
              Mensagem = "Desconhecido: "+ Mensagem_Recebida+"\n"
              
              conversa.append(Mensagem)
              lbl.config(text = conversa)

    except:
        if user['apelido'] != '':
          Mensagem = user['apelido'] +": "+ Mensagem_Recebida+"\n"
          
          conversa.append(Mensagem)
          lbl.config(text = conversa)
        else:
          Mensagem = "Desconhecido: "+ Mensagem_Recebida+"\n"
          
          conversa.append(Mensagem)
          lbl.config(text = conversa)


In [47]:
def interface():

    conversa = []

    def dadosMensagem():
        Mensagem = inputMensagem.get(1.0, "end-1c")
        if Mensagem != "":
            tcp.send(bytes(Mensagem,"utf8"))
            Mensagem = 'Eu: ' + Mensagem+"\n"
            conversa.append(Mensagem)
            lbl.config(text = conversa)

    def close():
        tcp.close()

    #Pegar o arquivo a ser enviado
    def procuraArquivo(): 
        filename = filedialog.askopenfilename(initialdir = "/", 
            title = "Select a File", 
            filetypes = (("all files", "*.*"),("all files", "*.*"))) 
        
        name = filename.split('/')
        file_size = os.path.getsize(filename)
        aviso = '#arquivo|'+name[-1]+'|'+str(file_size)
        tcp.send(bytes(aviso,"utf8"))

        ###############################################
        print(filename)

        arquivo = open (filename, "rb")
        ler_buffer = arquivo.read(1024) 
        while (ler_buffer):
            print("\renviando...")
            tcp.send(ler_buffer) 
            ler_buffer = arquivo.read(1024)
        tcp.send(bytes("#enviado","utf8"))
        print("arquivo enviado")
        ################################################

    root = Tk()

    inputMensagem = Text(root)
    inputMensagem["height"] = 2
    inputMensagem["width"] = 50
    inputMensagem["wrap"] = WORD
    inputMensagem.grid(column=0, row=0, padx=10, pady=10)
    inputMensagem.pack()

    lbl = Label(root)
    lbl["text"] = ""
    lbl.pack()

    mensagemButton = Button(root) 
    mensagemButton["text"] = "Enviar Mensagens"
    mensagemButton["command"] = dadosMensagem
    mensagemButton.pack()

    button_explore = Button(root)  
    button_explore["text"] = "Enviar Arquivo" 
    button_explore["command"] = procuraArquivo
    button_explore.pack()

    mensagemButton = Button(root) 
    mensagemButton["text"] = "Fechar"
    mensagemButton["command"] = close
    mensagemButton.pack()

    threading.Thread(target=recebendoMensagem, args=(lbl,conversa,)).start()
    root.mainloop()

In [48]:
interface()

tcp.close()
# finalizar o socket

Exception in thread Thread-10 (recebendoMensagem):
Traceback (most recent call last):
  File "C:\Users\ianha\AppData\Local\Temp\ipykernel_22496\1979602732.py", line 12, in recebendoMensagem
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ianha\AppData\Local\Temp\ipykernel_22496\1979602732.py", line 49, in recebendoMensagem
TypeError: can only concatenate str (not "bytes") to str


arquivo
